# MatKit Example 1: Truss Analysis with 1-Based Indexing

This example demonstrates how MatKit enables natural mathematical notation in FEM code using 1-based indexing.

In [1]:
import numpy as np
from matkit import Mesh, OneArray

## Define Mesh

Node coordinates and element connectivity using natural 1-based numbering.

In [2]:
# Node coordinates (mm)
P = [
    [0, 0],      # Node 1
    [500, 0],    # Node 2
    [300, 300],  # Node 3
    [600, 300],  # Node 4
]

# Element connectivity (1-based node numbers)
elements = [
    [1, 2],  # Element 1 connects nodes 1 and 2
    [1, 3],  # Element 2 connects nodes 1 and 3
    [2, 3],  # Element 3 connects nodes 2 and 3
    [2, 4],  # Element 4 connects nodes 2 and 4
    [3, 4],  # Element 5 connects nodes 3 and 4
]

# Create mesh (auto-detects ROD elements from 2 nodes per element)
mesh = Mesh(P, elements)
mesh.summary()

Mesh Summary
Element type:       ROD - 1D rod/truss element
Number of nodes:    4
Number of elements: 5
Dimension:          2D
DOFs per node:      2
Total DOFs:         8

Node Coordinates (1-based numbering):
------------------------------------------------------------
  Node 1: [0. 0.]
  Node 2: [500.   0.]
  Node 3: [300. 300.]
  Node 4: [600. 300.]

Element Connectivity (1-based numbering):
------------------------------------------------------------
  Element 1: Nodes [np.int64(1), np.int64(2)]
  Element 2: Nodes [np.int64(1), np.int64(3)]
  Element 3: Nodes [np.int64(2), np.int64(3)]
  Element 4: Nodes [np.int64(2), np.int64(4)]
  Element 5: Nodes [np.int64(3), np.int64(4)]


## FEM Results

Results from a truss analysis stored with 1-based indexing using OneArray.

In [3]:
# Displacement vector u (in mm)
# Format: [u1x, u1y, u2x, u2y, u3x, u3y, u4x, u4y]
u = OneArray([0., 0., 0.60047176, -0.6221258, 0.59000443, -1.25455763, 0., 0.])

# Element forces N (in N)
# Positive = tension, Negative = compression
N = OneArray([6052.76, -5582.25, -7274.51, 6380.16, -9912.07])

# Reaction forces (in N)
# Format: [R1x, R1y, R2x, R2y, R3x, R3y, R4x, R4y]
R = OneArray([-2105.51, 3947.24, 0., 0., 10000., -10000., -7894.49, 6052.76])

## Display Element Forces

Notice the clean 1-based access: `N[iel]` directly gives the force in element `iel`.

In [4]:
print("Element Forces:")
print("=" * 60)

for iel in mesh.element_numbers():
    node_nums, coords = mesh.get_element(iel)
    force = N[iel]  # Natural 1-based access!
    force_type = "Tension" if force > 0 else "Compression"
    
    print(f"Element {iel}: Nodes {node_nums[0]}-{node_nums[1]}, "
          f"N = {force:8.2f} N ({force_type})")

Element Forces:
Element 1: Nodes 1-2, N =  6052.76 N (Tension)
Element 2: Nodes 1-3, N = -5582.25 N (Compression)
Element 3: Nodes 2-3, N = -7274.51 N (Compression)
Element 4: Nodes 2-4, N =  6380.16 N (Tension)
Element 5: Nodes 3-4, N = -9912.07 N (Compression)


## Display Node Displacements

Access displacement components using DOF mapping.

In [5]:
print("\nNode Displacements:")
print("=" * 60)

for inode in mesh.node_numbers():
    dofs = mesh.dofs_for_node(inode)
    ux, uy = u.data[dofs]  # DOF indices are 0-based for array access
    magnitude = np.sqrt(ux**2 + uy**2)
    
    print(f"Node {inode}: u = [{ux:7.4f}, {uy:7.4f}] mm, |u| = {magnitude:.4f} mm")


Node Displacements:
Node 1: u = [ 0.0000,  0.0000] mm, |u| = 0.0000 mm
Node 2: u = [ 0.6005, -0.6221] mm, |u| = 0.8646 mm
Node 3: u = [ 0.5900, -1.2546] mm, |u| = 1.3864 mm
Node 4: u = [ 0.0000,  0.0000] mm, |u| = 0.0000 mm


## Display Reaction Forces

Show reaction forces at support nodes.

In [6]:
print("\nReaction Forces:")
print("=" * 60)

for inode in mesh.node_numbers():
    dofs = mesh.dofs_for_node(inode)
    Rx, Ry = R.data[dofs]  # DOF indices are 0-based for array access
    
    if abs(Rx) > 1e-6 or abs(Ry) > 1e-6:  # Only show non-zero reactions
        magnitude = np.sqrt(Rx**2 + Ry**2)
        print(f"Node {inode}: R = [{Rx:8.2f}, {Ry:8.2f}] N, |R| = {magnitude:.2f} N")


Reaction Forces:
Node 1: R = [-2105.51,  3947.24] N, |R| = 4473.69 N
Node 3: R = [10000.00, -10000.00] N, |R| = 14142.14 N
Node 4: R = [-7894.49,  6052.76] N, |R| = 9947.81 N


## Key Features Demonstrated

1. **1-Based Mesh Creation**: Nodes numbered 1, 2, 3, 4 naturally
2. **1-Based Element Connectivity**: Element 1 connects nodes 1-2 (not 0-1!)
3. **OneArray for Results**: `N[iel]` gives force in element `iel` directly
4. **Clean Iteration**: `mesh.element_numbers()` instead of `range(1, n+1)`
5. **DOF Mapping**: `mesh.dofs_for_node(inode)` handles the translation

**No off-by-one errors!** The code reads like mathematical notation.